In [48]:
from stravalib.client import Client
import pandas as pd
from geopy.distance import vincenty
from retrying import retry
import time
import numpy as np
from tqdm import tqdm

client = Client(access_token='76824abf6abf903eb3d8b0bde83625135c0be0ec')
athlete = client.get_athlete()

In [49]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_leaderboard(client,segment):
    total = client.get_segment_leaderboard(segment)
    friends = client.get_segment_leaderboard(segment,following=True)
    c = (total,friends)
    return c

@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_segment(client,segment):
    return client.get_segment(segment)

In [51]:
df = pd.read_csv('segoutput.csv',index_col=False)

home = (45.487803,-73.576913)
df['coordinates'] = zip(df['latitude'], df['longitude'])

biglist = df.T.to_dict().values()
for row in biglist:
    row['distance_from_home'] = vincenty(row['coordinates'], home).km

new_df = pd.DataFrame(biglist)
new_df = new_df[new_df['distance_from_home'] <= 2].sort_values(['distance_from_home'], ascending=[True])

smalllist = new_df.T.to_dict().values()

for row in tqdm(smalllist):
    
    time.sleep(5)
        
    row['segment_distance'] = float(retry_get_segment(client,row["segment_id"]).distance)
    
    both_leaderboards = retry_get_leaderboard(client,row["segment_id"]) 
    row['total_top_time'] = both_leaderboards[0][0].elapsed_time
    
    times = both_leaderboards[0][0].elapsed_time
    for x in both_leaderboards[0][1:]:
        times += x.elapsed_time
    times = times / len(both_leaderboards[0])
    row['average_total_top_time'] = times
    
    
    if not both_leaderboards[1]:
        row['friend_top_time'] = np.nan        
    else:
        row['friend_top_time'] = both_leaderboards[1][0].elapsed_time
        
final_df = pd.DataFrame(smalllist)
final_df['friend_versus_top1'] = (final_df['friend_top_time'] - final_df['total_top_time'])/final_df['total_top_time']*100
final_df['friend_versus_top10'] = (final_df['friend_top_time'] - final_df['average_total_top_time'])/final_df['average_total_top_time']*100


,average_total_top_time,color,coordinates,distance_from_home,friend_top_time,id,latitude,longitude,name,segment_distance,segment_id,segment_name,total_top_time,type,url,friend_versus_top1,friend_versus_top10
89,04:24:30.500000,8126C0,"(45.4963872861, -73.5578264762)",1.770942,00:01:50,1490,45.496387,-73.557826,Promenade Du Vieux Port Climb: [Yibing Wen],766.010,1238255,Promenade Du Vieux Port Climb,00:01:50,Yibing Wen,http://www.strava.com/segments/1238255/compare/,0.000000,-99.306890
91,00:01:09.266666,FF9233,"(45.4953850283, -73.5631321734)",1.367683,00:00:50,1503,45.495385,-73.563132,Montée Peel 1: [Mario Born],500.751,1222218,Montée Peel 1,00:00:42,Mario Born,http://www.strava.com/segments/1222218/compare/,19.047619,-27.815206
79,00:01:00.933333,FFEE33,"(45.493244, -73.575643)",0.612815,00:00:45,1460,45.493244,-73.575643,Seymour Avenue Climb: [Sasha Wloski],428.400,6119622,Seymour Avenue Climb,00:00:28,Sasha Wloski,http://www.strava.com/segments/6119622/compare/,60.714286,-26.148796
55,00:00:53.600000,FF9233,"(45.485758, -73.566265)",0.862893,00:00:43,1411,45.485758,-73.566265,Ça pue la marde de cheval: [Mario Born],453.100,4945088,Ça pue la marde de cheval,00:00:43,Mario Born,http://www.strava.com/segments/4945088/compare/,0.000000,-19.776119
32,00:00:31.800000,FF9233,"(45.482434, -73.585358)",0.889922,00:00:27,849,45.482434,-73.585358,Atwater Exit Climb: [Mario Born],127.800,3935214,Atwater Exit Climb,00:00:17,Mario Born,http://www.strava.com/segments/3935214/compare/,58.823529,-15.094340
24,00:00:28.200000,8126C0,"(45.504655, -73.571183)",1.925761,00:00:24,906,45.504655,-73.571183,Avenue Union S. (Maisonneuve/Cathcart): [Yibi...,277.000,8471089,Avenue Union S. (Maisonneuve/Cathcart),00:00:17,Yibing Wen,http://www.strava.com/segments/8471089/compare/,41.176471,-14.893617
53,00:01:56.600000,FF9233,"(45.488617, -73.570898)",0.478843,00:01:41,1409,45.488617,-73.570898,Avenue Lionel-Groulx O. (Lionel-Groulx / Atwat...,890.500,9911222,Avenue Lionel-Groulx O. (Lionel-Groulx / Atwater),00:01:41,Mario Born,http://www.strava.com/segments/9911222/compare/,0.000000,-13.379074
31,00:00:20.500000,FFEE33,"(45.482616, -73.591666)",1.289418,00:00:18,848,45.482616,-73.591666,Avenue Kensington N. (Sainte-Catherine/Maisonn...,177.400,8509610,Avenue Kensington N. (Sainte-Catherine/Maisonn...,00:00:15,Sasha Wloski,http://www.strava.com/segments/8509610/compare/,20.000000,-12.195122
30,00:00:42.066666,FF9233,"(45.48241, -73.575363)",0.611511,00:00:37,1358,45.482410,-73.575363,Rue Charlevoix (Bridge Sprint): [Mario Born],362.379,3626751,Rue Charlevoix (Bridge Sprint),00:00:32,Mario Born,http://www.strava.com/segments/3626751/compare/,15.625000,-12.044373
49,00:00:34.466666,FF9233,"(45.487818, -73.582785)",0.459046,00:00:32,1399,45.487818,-73.582785,Atwater Downhill Extended: [Mario Born],492.400,8204306,Atwater Downhill Extended,00:00:25,Mario Born,http://www.strava.com/segments/8204306/compare/,28.000000,-7.156671


In [53]:
print final_df.sort_values(['friend_versus_top10'], ascending=[False])

    average_total_top_time   color                      coordinates  \
1          00:00:23.500000  A03D3D  (45.5021951906, -73.5629598051)   
61                00:00:17  FFEE33          (45.490744, -73.594655)   
0          00:00:22.500000  FFEE33          (45.502192, -73.577163)   
74         00:00:36.100000  FFEE33          (45.492301, -73.598682)   
77         00:01:52.100000  FFEE33  (45.4925382324, -73.5984772816)   
69                00:00:36  B7E628  (45.4903114121, -73.5572500527)   
6          00:00:14.500000  A03D3D          (45.505209, -73.570691)   
44         00:00:14.100000  FFEE33          (45.490763, -73.596486)   
51         00:00:20.200000  FFEE33  (45.5003406946, -73.5856961366)   
56         00:00:18.600000  C7258E          (45.477057, -73.571581)   
20                00:01:12  FFEE33          (45.483232, -73.579054)   
15         00:00:16.500000  C7258E          (45.472359, -73.569521)   
16         00:04:42.700000  FFEE33          (45.488943, -73.594291)   
7     

In [43]:
final_df

,average_total_top_time,color,coordinates,distance_from_home,friend_top_time,id,latitude,longitude,name,segment_distance,segment_id,segment_name,total_top_time,type,url
0,00:00:22.500000,FFEE33,"(45.502192, -73.577163)",1.599334,00:00:58,1537,45.502192,-73.577163,Peel Push - Sherbrooke to Docteur Penfield : ...,204.300,7849363,Peel Push - Sherbrooke to Docteur Penfield,00:00:19,Sasha Wloski,http://www.strava.com/segments/7849363/compare/
1,00:00:23.500000,A03D3D,"(45.5021951906, -73.5629598051)",1.936012,00:01:14,1538,45.502195,-73.562960,Beaver Hall: [Laurie McQueen],237.550,1176830,Beaver Hall,00:00:20,Laurie McQueen,http://www.strava.com/segments/1176830/compare/
2,00:00:51.400000,FFEE33,"(45.4916093499, -73.5882798583)",0.984138,00:01:25,1539,45.491609,-73.588280,Atwater/Saint Sulpice Short Safe: [Sasha Wloski],411.851,1840013,Atwater/Saint Sulpice Short Safe,00:00:40,Sasha Wloski,http://www.strava.com/segments/1840013/compare/
3,00:00:58.400000,9CD0EF,"(45.49695, -73.586141)",1.246526,00:01:15,1540,45.496950,-73.586141,DrPenfield: [Eoin Craigie],836.300,7381906,DrPenfield,00:00:53,Eoin Craigie,http://www.strava.com/segments/7381906/compare/
